# Introduction to Transformations
This example introduces the `pybop.BaseTransformation` class and its instances. This class enables the cost and likelihood functions to be evaluated in a transformed search space. The search space is used by the optimiser and sampler classes during inference. These transformations can be both linear (e.g. `pybop.ScaledTransformation`) and non-linear (e.g. `pybop.LogTransformation`). By default, if transformations are applied, the sampling and optimisers will search in the transformed space.

Transformations can be helpful when the difference in parameter magnitudes is large, or to create a search space that is better posed for the optimisation algorithm. Before we begin, we need to ensure that we have all the necessary tools. We will install and import PyBOP alongside any other package dependencies.

In [14]:
%pip install --upgrade pip ipywidgets -q
%pip install pybop -q

import numpy as np
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


First, to showcase the transformation functionality, we need to construct a cost. This class is typically built on the following objects:
- Model
- Dataset
- Parameters to identify
- Problem

We will first construct the model, then the parameters and corresponding dataset. Once that is complete, the problem will be created. With the cost class created, we will showcase the different interactions users can have with transformations.

In [15]:
model = pybamm.lithium_ion.SPM()

Now that we have the model constructed, let's define the parameters for identification. At this point, we define the transformations applied to each parameter. PyBOP allows for transformations to be applied at the individual parameter level, which are then combined for application during the optimisation. Below we apply a linear transformation using the `pybop.ScaledTransformation` class. This class has arguments for a `coefficient` which defines the linear stretch or scaling of the search space, and `intercept` which defines the translation or shift. The equation for this transformation is:

$$
y_{search} = m(x_{model}+b)
$$

where $m$ is the linear scale coefficient, $b$ is the intercept, $x_{model}$ is the model parameter space, and $y_{search}$ is the transformed space.

In [16]:
parameters = pybop.Parameters([
    pybop.Parameter(
        "Negative electrode active material volume fraction",
        initial_value=0.6,
        bounds=[0.35, 0.7],
        transformation=pybop.ScaledTransformation(
            coefficient=1 / 0.35, intercept=-0.35, n_parameters=1
        ),
    ),
    pybop.Parameter(
        "Positive electrode active material volume fraction",
        initial_value=0.6,
        bounds=[0.45, 0.625],
        transformation=pybop.ScaledTransformation(
            coefficient=1 / 0.175, intercept=-0.45, n_parameters=1
        ),
    ),
])

Next, to create the `pybop.Dataset` we generate some synthetic data from the model using the `model.predict` method.

In [17]:
t_eval = np.linspace(0, 10, 100)
sim = pybamm.Simulation(model)
sol = sim.solve(t_eval=t_eval)

dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": sol["Current [A]"](t_eval),
        "Voltage [V]": sol["Voltage [V]"](t_eval),
    }
)

Now that we have the model, parameters, and dataset, we can combine them and construct the problem class.

In [18]:
builder = (
    pybop.builders.Pybamm()
    .set_dataset(dataset)
    .set_simulation(
        model,
    )
    .add_cost(
        pybop.costs.pybamm.SumOfPower("Voltage [V]", "Voltage [V]")
    )
)
for param in parameters:
    builder.add_parameter(param)

problem = builder.build()

Now let us see how we can get the cost value for an input.

In [19]:
problem.set_params([0.6, 0.6])
cost_value = problem.run()
print("Cost:", cost_value)

Cost: [0.00690259]


Now let us set up and run the optimisation process.

In [20]:
options = pybop.ScipyMinimizeOptions(
    maxiter=50,
    method="Nelder-Mead",
    tol=1e-9,
    verbose=True,
 )
optim = pybop.SciPyMinimize(problem, options=options)
results = optim.run()

Iter: 1 | Evals: 5 | Best Values: [0.57575 0.5445 ] | Best Cost: 0.001380206811561706 |
Iter: 2 | Evals: 7 | Best Values: [0.562625 0.53925 ] | Best Cost: 0.0009843043131614862 |
Iter: 3 | Evals: 9 | Best Values: [0.5875625 0.515625 ] | Best Cost: 0.00017411497797826202 |
Iter: 4 | Evals: 11 | Best Values: [0.5744375 0.510375 ] | Best Cost: 5.094527944788734e-05 |
Iter: 5 | Evals: 12 | Best Values: [0.5744375 0.510375 ] | Best Cost: 5.094527944788734e-05 |
Iter: 6 | Evals: 14 | Best Values: [0.58723438 0.50709375] | Best Cost: 2.9002043035206324e-05 |
Iter: 7 | Evals: 16 | Best Values: [0.59010547 0.49774219] | Best Cost: 9.39636923924622e-06 |
Iter: 8 | Evals: 17 | Best Values: [0.59010547 0.49774219] | Best Cost: 9.39636923924622e-06 |
Iter: 9 | Evals: 19 | Best Values: [0.59186914 0.50159766] | Best Cost: 5.273101261618547e-07 |
Iter: 10 | Evals: 20 | Best Values: [0.59186914 0.50159766] | Best Cost: 5.273101261618547e-07 |
Iter: 50 | Evals: 90 | Best Values: [0.6086141  0.49917649]

We can see that the result of the two cost evaluations (with and without transformations) by comparing the output.

In [ ]:
# Compare cost evaluations with and without transformations
problem.set_params(parameters.get_initial_values(transformed=True))
cost_transformed = problem.run()
problem.set_params(parameters.get_initial_values(transformed=False))
cost_untransformed = problem.run()
cost_transformed == cost_untransformed
# TODO: investigate why these are not equal

array([False])

We can also compare the cost landscapes plotted in the model and search spaces. Let's first plot the cost in the model space through the conventional method:

In [22]:
pybop.plot.contour(problem);

Next, we can use the `apply_transform` argument when constructing the cost landscape to plot in the transformed space.

In [23]:
pybop.plot.contour(problem, steps=15, apply_transform=True);

Note the difference in axis scale compared to the non-transformed landscape. Next, let's change the transformation on the 'Positive electrode active material volume fraction' to a non-linear log space.

In [ ]:
# Replace the parameter with a new one using LogTransformation
old_param = parameters.remove("Positive electrode active material volume fraction")
new_param = pybop.Parameter(
    "Positive electrode active material volume fraction",
    initial_value=old_param.initial_value,
    bounds=old_param.bounds,
    transformation=pybop.LogTransformation(n_parameters=1)
 )
parameters.add(new_param)
transform = parameters.transformation

Let's update the bounds and plot the cost landscape again. This time, the values on the y-axis are negative as they correspond to the log of the model values.

In [33]:
pybop.plot.contour(problem, steps=15, apply_transform=True);

## Concluding Thoughts

In this notebook, we've introduced the transformation class and its interaction with the `pybop.Parameters` and `pybop.CostInterface` classes. Transformations allow the optimisation or sampling search space to be transformed for improved convergence in situations where the optimisation hyperparameters are poorly tuned, or in optimisation tasks with high variance in the parameter magnitudes. 